<a href="https://colab.research.google.com/github/KimNikita/machine-learning-practice/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import sklearn

In [ ]:
url = "https://raw.githubusercontent.com/KimNikita/machine-learning-practice/main/anime-dataset-2023.csv"
data_raw = pd.read_csv(url)

In [ ]:
data_raw.shape

(24905, 24)

In [ ]:
data_raw.head()

,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
0,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",...,Sunrise,Original,24 min per ep,R - 17+ (violence & profanity),41.0,43,78525,914193.0,1771505,https://cdn.myanimelist.net/images/anime/4/196...
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,"Sep 1, 2001",...,Bones,Original,1 hr 55 min,R - 17+ (violence & profanity),189.0,602,1448,206248.0,360978,https://cdn.myanimelist.net/images/anime/1439/...
2,6,Trigun,Trigun,トライガン,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26.0,"Apr 1, 1998 to Sep 30, 1998",...,Madhouse,Manga,24 min per ep,PG-13 - Teens 13 or older,328.0,246,15035,356739.0,727252,https://cdn.myanimelist.net/images/anime/7/203...
3,7,Witch Hunter Robin,Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),7.25,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,TV,26.0,"Jul 3, 2002 to Dec 25, 2002",...,Sunrise,Original,25 min per ep,PG-13 - Teens 13 or older,2764.0,1795,613,42829.0,111931,https://cdn.myanimelist.net/images/anime/10/19...
4,8,Bouken Ou Beet,Beet the Vandel Buster,冒険王ビィト,6.94,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,TV,52.0,"Sep 30, 2004 to Sep 29, 2005",...,Toei Animation,Manga,23 min per ep,PG - Children,4240.0,5126,14,6413.0,15001,https://cdn.myanimelist.net/images/anime/7/215...


#Удаление ненужных столбцов

In [ ]:
data = data_raw.drop(data_raw.columns[[0, 1, 2, 3, 6, 9, 12, 13, 16, 18, 19, 20, 22, 23]], axis=1)
data.head()

,Score,Genres,Type,Episodes,Premiered,Status,Studios,Source,Rating,Scored By
0,8.75,"Action, Award Winning, Sci-Fi",TV,26.0,spring 1998,Finished Airing,Sunrise,Original,R - 17+ (violence & profanity),914193.0
1,8.38,"Action, Sci-Fi",Movie,1.0,UNKNOWN,Finished Airing,Bones,Original,R - 17+ (violence & profanity),206248.0
2,8.22,"Action, Adventure, Sci-Fi",TV,26.0,spring 1998,Finished Airing,Madhouse,Manga,PG-13 - Teens 13 or older,356739.0
3,7.25,"Action, Drama, Mystery, Supernatural",TV,26.0,summer 2002,Finished Airing,Sunrise,Original,PG-13 - Teens 13 or older,42829.0
4,6.94,"Adventure, Fantasy, Supernatural",TV,52.0,fall 2004,Finished Airing,Toei Animation,Manga,PG - Children,6413.0


#Удаление ненужных строк

In [ ]:
data = data.loc[data['Scored By'] != 'UNKNOWN']
data['Scored By'] = data['Scored By'].astype('float')
data['Scored By'].quantile([0.005,.01,.05,.1,.5,.6,.7,.8,.9,.95,.99,.995])

<ipython-input-107-9969061a36b3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Scored By'] = data['Scored By'].astype('float')


0.005       108.00
0.010       112.00
0.050       137.00
0.100       174.00
0.500      1760.00
0.600      3261.00
0.700      6875.40
0.800     17910.00
0.900     59841.30
0.950    137178.05
0.990    538387.40
0.995    839502.10
Name: Scored By, dtype: float64

In [ ]:
data.shape

(15692, 10)

In [ ]:
data = data.loc[data['Scored By'] >= data['Scored By'].quantile(0.8)]

data = data.loc[data['Episodes'] != '1.0']
data = data.loc[data['Type'].isin(['ONA', 'TV', 'UNKNOWN'])]
data = data.loc[data['Source'].isin(['Original', 'Manga' 'Light novel', '4-koma manga', 'Visual novel', 'Other', 'Novel', 'Game', 'Book', 'Web manga', 'Mixed media', 'Card game', 'Web novel'])]

data = data.loc[data['Status'] == 'Finished Airing']
data = data.drop(data.columns[5], axis=1)

#Преобразование типов и данных

In [ ]:
data['Type'] = data['Type'].astype('category')
data['Source'] = data['Source'].astype('category')

# TODO сначала преобразование
#data['Premiered'] = data['Premiered'].astype('category')
#data['Studios'] = data['Studios'].astype('category')
#data['Rating'] = data['Rating'].astype('category')

# TODO разбиение
#data['Genres']

data['Score'] = data['Score'].astype('float')
data['Episodes'] = data['Episodes'].astype('float')

# Уникальные значения

In [ ]:
print(data['Genres'].unique())
print(data['Type'].unique())
print(data['Episodes'].unique())
print(data['Premiered'].unique())
print(data['Studios'].unique())
print(data['Source'].unique())
print(data['Rating'].unique())

['Action, Award Winning, Sci-Fi'
 'Action, Avant Garde, Award Winning, Drama, Sci-Fi, Suspense'
 'Adventure, Fantasy, Mystery' 'Comedy, Slice of Life'
 'Comedy, Drama, Fantasy, Romance, Ecchi' 'Action, Drama, Sci-Fi'
 'Action, Award Winning, Drama, Romance, Sci-Fi'
 'Action, Drama, Romance, Sci-Fi' 'Action, Adventure, Sci-Fi'
 'Drama, Romance, Supernatural' 'Comedy, Romance, Ecchi' 'Drama, Romance'
 'Action, Drama, Horror, Mystery, Supernatural' 'Sci-Fi'
 'Action, Horror, Mystery, Romance, Supernatural'
 'Adventure, Fantasy, Romance' 'Drama, Romance, Sci-Fi'
 'Action, Adventure, Drama, Fantasy, Mystery, Sci-Fi'
 'Action, Adventure, Comedy' 'Horror, Mystery, Supernatural'
 'Adventure, Drama, Romance, Sci-Fi'
 'Award Winning, Drama, Mystery, Sci-Fi, Supernatural, Suspense'
 'Avant Garde, Drama, Mystery, Supernatural, Suspense'
 'Avant Garde, Award Winning, Drama, Mystery, Sci-Fi, Supernatural'
 'Action, Fantasy, Romance, Supernatural' 'Drama, Fantasy, Mystery'
 'Award Winning, Comedy, Dr

In [ ]:
data.shape

(792, 9)

# TODO
## Переделать отброс по квантилям
## Провести калибровку Score с учетом Scored By
## Remove dark side
## Разбить значения в Genres
## Разбить значения в Studios
## Разбить значения в Premiered
## Разбить значения в Rating
## Отфильтровать Source